**Table of Contents**
<div id="toc"></div>

# IMPORTS

In [1]:
from ete3 import Tree
import collections
from collections import Counter
from Bio import AlignIO
from Bio import SeqIO
import numpy as np

# RAXML TREE

In [16]:
fw=open('spliced_cds_work/EBOV_CDS_splice_39_seqs_underscores.fa','w')
with open('spliced_cds_work/EBOV_CDS_splice_39_seqs.fa','r') as f:
    for l in f:
        l=l.rstrip('\n')
        l=l.replace('|','_')
        fw.write(l+'\n')
fw.close()            

In [20]:
%%bash
cd spliced_cds_work/
raxmlHPC-PTHREADS -f a -x 12345 -p 12345 -\# 1000 -m GTRGAMMA -s EBOV_CDS_splice_39_seqs_underscores.fa -n raxml_rapid_spliced_cds_tree -o EBOV_AF272001_Mayinga_Yambuku_DRC_1976,EBOV_KC242801_deRoover_DRC_1976,EBOV_KC242791_Bonduni_Tandala_DRC_1977 -T 4


RAxML can't, parse the alignment file as phylip file 
it will now try to parse it as FASTA file



Bootstrap[997]: Time 0.162552 seconds, bootstrap likelihood -25060.890666, best rearrangement setting 15
Bootstrap[998]: Time 0.172141 seconds, bootstrap likelihood -24921.542504, best rearrangement setting 5
Bootstrap[999]: Time 0.149512 seconds, bootstrap likelihood -24833.443591, best rearrangement setting 6


Overall Time for 1000 Rapid Bootstraps 167.961885 seconds
Average Time per Rapid Bootstrap 0.167962 seconds

Starting ML Search ...

Fast ML optimization finished

Fast ML search Time: 62.350852 seconds

Slow ML Search 0 Likelihood: -24971.957508
Slow ML Search 1 Likelihood: -24971.957508
Slow ML Search 2 Likelihood: -24971.957514
Slow ML Search 3 Likelihood: -24971.957518
Slow ML Search 4 Likelihood: -24971.957512
Slow ML Search 5 Likelihood: -24971.957528
Slow ML Search 6 Likelihood: -24971.957527
Slow ML Search 7 Likelihood: -24971.957525
Slow ML Search 8 Likelihood: -24971.9

# ANCESTRAL RECONSTRUCTION


In [21]:
%%bash
cd spliced_cds_work/
raxmlHPC-PTHREADS -f A -t RAxML_bipartitionsBranchLabels.raxml_rapid_spliced_cds_tree -s EBOV_CDS_splice_39_seqs_underscores.fa -m GTRGAMMA -n ANC_RECON_CDS_splice -T 3


RAxML can't, parse the alignment file as phylip file 
it will now try to parse it as FASTA file



Found 5 sequences that are exactly identical to other sequences in the alignment.
Normally they should be excluded from the analysis.

An alignment file with sequence duplicates removed has already
been printed to file EBOV_CDS_splice_39_seqs_underscores.fa.reduced


Using BFGS method to optimize GTR rate parameters, to disable this specify "--no-bfgs" 


This is the RAxML Master Pthread

This is RAxML Worker Pthread Number: 1

This is RAxML Worker Pthread Number: 2


This is RAxML version 8.2.11 released by Alexandros Stamatakis on June 2017.

With greatly appreciated code contributions by:
Andre Aberer      (HITS)
Simon Berger      (HITS)
Alexey Kozlov     (HITS)
Kassian Kobert    (HITS)
David Dao         (KIT and HITS)
Sarah Lutteropp   (KIT and HITS)
Nick Pattengale   (Sandia)
Wayne Pfeiffer    (SDSC)
Akifumi S. Tanabe (NRIFS)
Charlie Taylor    (UF)


Alignment has 305 distinct align

In [22]:
fw = open('spliced_cds_work/RAxML_marginalAncestralStates.ANC_RECON_CDS_splice.fasta','w')
c=0
with open('spliced_cds_work/RAxML_marginalAncestralStates.ANC_RECON_CDS_splice','r') as f:
    for l in f:
        l=l.rstrip('\n')
        tokens = l.split(' ')
        fw.write('>'+tokens[0]+'\n'+tokens[1]+'\n')
        c+=1
fw.close()
print(c)
        

38


In [31]:
%%bash
cd spliced_cds_work/
cat RAxML_marginalAncestralStates.ANC_RECON_CDS_splice.fasta EBOV_CDS_splice_39_seqs_underscores.fa > EBOV_CDS_splice_with_marginalAncestralStates.fasta
grep -c '>' EBOV_CDS_splice_with_marginalAncestralStates.fasta

77


# FUNCTION DEFINITIONS

In [61]:
# FUNCTION DEFINITIONS #

def check_and_count(tree, old_count, snp, major_variant, minor_variant, minor_freq):
    check = minor_major_monophyly_check(tree, snp, major_variant, minor_variant, minor_freq)
    return tuple(map(lambda x, y: x + y, old_count, check))

def minor_major_monophyly_check(tree, snp, major_variant, minor_variant, minor_freq):
#     print(snp, major_variant, minor_variant)
    count = (0,0,0,0)
    if minor_freq==1:
        snp_classifier['only_occurs_once'].append((snp, major_variant+ ':'+minor_variant))
        count = (1,0,0,0)
    else:
        minor_monophyly_check = tree.check_monophyly(values=[minor_variant], target_attr="snp") 
        if minor_monophyly_check[0]:
            snp_classifier['minor_monophyly'].append((snp, major_variant+ ':'+minor_variant))
            count = (0,1,0,0)
        elif not minor_monophyly_check[0]:
            major_monophyly_check = tree.check_monophyly(values=[major_variant], target_attr="snp")
            if major_monophyly_check[0]:
                snp_classifier['major_monophyly'].append((snp, major_variant+ ':'+minor_variant))
                count = (0,0,1,0)
            elif not major_monophyly_check[0]:
                snp_classifier['not_monophyly'].append((snp, major_variant + ':'+minor_variant))
                count = (0,0,0,1)
    return count

# CREATING THE SNP DICT 


In [62]:
# CREATING THE SNP DICT #
snp_dict = {}

a = AlignIO.read("spliced_cds_work/EBOV_CDS_splice_with_marginalAncestralStates.fasta","fasta")
aa = np.array([list(rec) for rec in a], np.character, order="F")

for i in range(len(a[0])): #for each base up to the length of the alignment
    bp = aa[:, i:i+1] 
    bases = []
    for item in bp:
        bases.append(item[0])
    c =1
    if i < 100 or i > 5000: #18880 for full genome
        for ambig in ['N','?','-']:
            if ambig in bases:
                c +=1
        if len(set(bases))>c:
            snp_dict['snp_'+str(i+1)]=bases
    elif 'N' in bases:
        if len(set(bases))>2:
            snp_dict['snp_'+str(i+1)]=bases
    else:
        if len(set(bases))>1:
            snp_dict['snp_'+str(i+1)]=bases


# CREATING THE INDEX DICT

In [63]:
# CREATING THE INDEX DICT #
# LETS YOU KNOW WHICH ITEM IN THE SNP DICT VALUE (I.E. BASES LIST) CORRESPONDS TO WHICH RECORD IN THE ALIGNMENT #
index_dict = {}
c=0
for record in a:
    c +=1
    index_dict[record.id]= c

# COUNTING AND CLASSIFYING SNPS

In [64]:
# DEFINING COUNTERS #
simple_cases_counter = 0
complex_cases_counter = 0
early_gap =0
late_gap =0
ambiguous_base=0

SIMPLE = (0,0,0,0)
COMPLEX = (0,0,0,0)

# CLASSIFYING THE SNPS #

snp_classifier = collections.defaultdict(list)

for snp in sorted(snp_dict, key= lambda x: float(x.split('_')[1])): 
    snp_count = Counter()
    
    t = Tree("spliced_cds_work/RAxML_nodeLabelledRootedTree.ANC_RECON_CDS_splice",format=8)
    for node in t.traverse("preorder"):
        if node.is_leaf():
            node.add_features(snp=snp_dict[snp][index_dict[node.name]-1])  #add snp annotations to each tip
            snp_count[snp_dict[snp][index_dict[node.name]-1]]+=1 #count the leaf snp frequency
            
    snp_count = snp_count.most_common()   
    
    major_variant = snp_count[0][0]

    # SIMPLE CASE #
    if len(snp_count)==2:
        # e.g. snp_189 [('G', 68), ('A', 4)]
        simple_cases_counter +=1 
        
        minor_variant = snp_count[1][0]
        minor_freq = snp_count[1][1]
        
        if minor_variant != 'N' and minor_variant != '?':
            SIMPLE = check_and_count(t, SIMPLE, snp, major_variant, minor_variant, minor_freq)            
    else:
        # COMPLEX CASE #
        # eg. snp_18913 [('G', 68), ('N', 3), ('A', 2), ('R', 2), ('-', 2)] #
        complex_cases_counter +=1 
        
        for k in snp_count[1:]:
#             print(k)
            minor_variant = k[0]
            minor_freq = k[1]
        
#             if float(snp.split('_')[1]) < 100 and k[0]=='-':
#                 early_gap += 1
            if k[0]=='N' or k[0]=='?':
                ambiguous_base += 1
            elif float(snp.split('_')[1]) > 5000:
#                 if float(snp.split('_')[1]) > 18900 and k[0]=='-':
#                     late_gap +=1
#                 else:
                new_id_list = [i for i in index_dict.keys() if i not in ['EBOV_JQ352763_Kikwit_Kikwit_DRC_1995-05-04','EBOV_HQ613403_M-M_DRC_2007-08-31','EBOV_HQ613402_034-KS_DRC_2008-12-31']]    
                t.prune(new_id_list)
                COMPLEX = check_and_count(t, COMPLEX, snp, major_variant, minor_variant, minor_freq)  
            else:
                COMPLEX = check_and_count(t, COMPLEX, snp, major_variant, minor_variant, minor_freq) 



# RESULT PRINT OUT

In [65]:
# RESULT PRINT OUT #
print '##### RESULTS #####'
print 'Simple, Complex, Total: ',simple_cases_counter, '+', complex_cases_counter,'=', simple_cases_counter+complex_cases_counter
print '\n'
print '##### SNP Classifier Dict #####'
for k in snp_classifier:
    key_dict = collections.defaultdict(list)
    for i in snp_classifier[k]:
        key_dict[i[0]].append(i[1])
    print k,'(All:', len(snp_classifier[k]), ', Unique:', len(key_dict), ')'
print '\n'    
print '##### Simple cases #####'
print '(only a single snp, minor monophyly, major monophyly, not monophyletic)'
print SIMPLE
print '\n'        
print '##### Not quite as simple cases #####'
print 'minor N or ? ', ambiguous_base
print 'early - ', early_gap
print 'late - ', late_gap
print '(only a single snp, minor monophyly, major monophyly, not monophyletic)'
print COMPLEX
print '\n'   
print '##### Tally of the two #####'
print '(', SIMPLE[0]+COMPLEX[0], SIMPLE[1]+COMPLEX[1], SIMPLE[2]+COMPLEX[2], SIMPLE[3]+COMPLEX[3], ')'

##### RESULTS #####
Simple, Complex, Total:  340 + 514 = 854


##### SNP Classifier Dict #####
only_occurs_once (All: 159 , Unique: 157 )
not_monophyly (All: 51 , Unique: 51 )
major_monophyly (All: 58 , Unique: 58 )
minor_monophyly (All: 599 , Unique: 591 )


##### Simple cases #####
(only a single snp, minor monophyly, major monophyly, not monophyletic)
(62, 239, 18, 21)


##### Not quite as simple cases #####
minor N or ?  507
early -  0
late -  0
(only a single snp, minor monophyly, major monophyly, not monophyletic)
(97, 360, 40, 30)


##### Tally of the two #####
( 159 599 58 51 )


# GENE LOCATION DICT

In [66]:
gene={}
for i in range(1,2221):
    gene[i]="NP"
for i in range(2221,3244):
    gene[i]="VP35"
for i in range(3244,4225):
    gene[i]="VP40"
for i in range(4225,5321):
    gene[i]="GP"
for i in range(5321,6188):
    gene[i]="VP30"
for i in range(6188,6944):
    gene[i]="VP24"
for i in range(6944,13583):
    gene[i]="L"
print(len(gene))

13582


In [68]:
fw=open('spliced_cds_work/identify_and_classify_snps_results.csv','w')
fw.write('Type,SNP,Major,Minor,Position,Gene,Position_in_Codon\n')
for k in snp_classifier:
    for i in snp_classifier[k]:
        location = int(i[0].split('_')[1])
        fw.write('{},{},{},{},{},{},{}\n'.format(k, i[1], i[1].split(':')[0], i[1].split(':')[1], location, gene[location], location%3))
fw.close()